## <center style="color:red;">**BrainScan – Détection du Cancer Cérébral par CNN**</center>

<br>

### <span style="color:green;">**Conception du Modèle CNN :**</span>

#### <span style="color:blue;">**1. Définir l’architecture du modèle CNN :**</span>

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model
import time
import joblib as jbl

variables = jbl.load("../util/variables.pkl")

X_train = variables["X_train"]
y_train = variables["y_train"]
X_test = variables["X_test"]
y_test = variables["y_test"]
images = variables["images"]
labels = variables["labels"]

print("Variables importées avec succès !")

Variables importées avec succès !


**Créer le modèle avec des couches Conv2D, MaxPooling, Dropout, Dense :**

In [ ]:

model = models.Sequential([
    # Couche 1 : Convolution + ReLU + MaxPooling
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    
    # Couche 2 : Convolution + ReLU + MaxPooling
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Couche 3 : Convolution + ReLU + MaxPooling
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Dropout pour éviter le surapprentissage
    layers.Dropout(0.3),
    
    # Couche Flatten pour aplatir les cartes de caractéristiques
    layers.Flatten(),
    
    # Couche Dense cachée
    layers.Dense(128, activation='relu'),
    
    # Dropout supplémentaire
    layers.Dropout(0.3),
    
    # Couche de sortie (nombre de neurones = nombre de classes)
    layers.Dense(y_train.shape[1], activation='softmax')  # Softmax pour classification multi-classes
])